In [47]:
!pip install pandas

In [48]:
import pandas as pd

df = pd.read_csv('budgetwise_finance_dataset.csv')

print(df.head())
print("\n ________________________________________________INFO__________________________________________ \n")
print(df.info()) 


  transaction_id user_id        date transaction_type       category amount  \
0          T4999    U018  2023-04-25          Expense       Educaton   3888   
1         T12828    U133  08/05/2022          Expense           rent    649   
2          T7403    U091    31-12-23           Income      Freelance  13239   
3         T12350    U097         NaN          Expense            Fod   6299   
4          T7495    U088  10/28/2022          Expense  entertainment   2287   

    payment_mode   location             notes  
0           card  Ahmedabad     Movie tickets  
1            NaN  Hyderabad            asdfgh  
2            Csh        BAN             Books  
3  Bank Transfer  AHMEDABAD  Electricity bill  
4           CARD  Hyderabad               NaN  

 ________________________________________________INFO__________________________________________ 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15900 entries, 0 to 15899
Data columns (total 9 columns):
 #   Column            Non-Nul

In [49]:

print(raw_dates['date'].unique()[:20])


['2023-04-25' '08/05/2022' '31-12-23' nan '10/28/2022' '04/11/2024'
 '08/16/2022' '2024-12-12' '13-03-24' '11/10/2023' '2022-04-15'
 '2021-04-18' '06/29/2022' '02/24/2022' '08/28/2023' '02-06-23' '17-07-24'
 '2021-02-27' '21-09-2024' '25-02-23']


In [50]:
from datetime import datetime

date_formats = [
    "%Y-%m-%d",  
    "%m/%d/%Y",  
    "%d-%m-%y",  
    "%d-%m-%Y",  
]

def parse_flexible_date(date_str):
    for fmt in date_formats:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            continue
    return pd.NaT

df['date'] = df['date'].apply(lambda x: parse_flexible_date(str(x)) if pd.notna(x) else pd.NaT)

df = df[df['date'].notna()].reset_index(drop=True)

print(df.head())
print("\n ________________________________________________INFO__________________________________________ \n")
print(df.info())


  transaction_id user_id       date transaction_type       category amount  \
0          T4999    U018 2023-04-25          Expense       Educaton   3888   
1         T12828    U133 2022-08-05          Expense           rent    649   
2          T7403    U091 2023-12-31           Income      Freelance  13239   
3          T7495    U088 2022-10-28          Expense  entertainment   2287   
4         T12465    U042 2024-04-11          Expense          Foods   4168   

  payment_mode   location          notes  
0         card  Ahmedabad  Movie tickets  
1          NaN  Hyderabad         asdfgh  
2          Csh        BAN          Books  
3         CARD  Hyderabad            NaN  
4          NaN        NaN           test  

 ________________________________________________INFO__________________________________________ 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15414 entries, 0 to 15413
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------ 

In [51]:
category_corrections = {
    'Educaton': 'Education',
    'education': 'Education',
    'EDU': 'Education',
    'Edu': 'Education',

    'rent': 'Rent',
    'Rentt': 'Rent',
    'RENT': 'Rent',
    'Rnt': 'Rent',

    'Foods': 'Food',
    'FOOD': 'Food',
    'food': 'Food',
    'Foodd': 'Food',
    'Fod': 'Food',

    'entertainment': 'Entertainment',
    'Entertainment': 'Entertainment',
    'Entrtnmnt': 'Entertainment',
    'Entertain': 'Entertainment',

    'Utilties': 'Utilities',
    'Utility': 'Utilities',
    'Utlities': 'Utilities',
    'utilities': 'Utilities',
    'Utilities': 'Utilities',

    'Travel': 'Travel',
    'Travl': 'Travel',
    'Traval': 'Travel',
    'travel': 'Travel',
    'TRAVEL': 'Travel',

    'HEALTH': 'Health',
    'Helth': 'Health',
    'health': 'Health',
    'Health': 'Health',

    'savings': 'Savings',
    'Saving': 'Savings',
    'SAVINGS': 'Savings',
    'Savings': 'Savings',

    'Others': 'Others',
    'others': 'Others',
    'Other': 'Others',
    'Misc': 'Others',

}

df['category'] = df['category'].replace(category_corrections)

df['category'] = df['category'].str.title()

df['category'] = df['category'].fillna('Others')

print("Cleaned categories:", df['category'].unique())


Cleaned categories: ['Education' 'Rent' 'Freelance' 'Entertainment' 'Food' 'Salary'
 'Utilities' 'Others' 'Travel' 'Investment' 'Health' 'Savings' 'Bonus']


In [52]:

payment_mode_corrections = {
    'card': 'Card',
    'CARD': 'Card',
    'Crd': 'Card',
    'CRD': 'Card',
    'csh': 'Cash',
    'Csh': 'Cash',
    'cash': 'Cash',
    'Cash': 'Cash',
    'CASH': 'Cash',
    'upi': 'UPI',
    'UPI': 'UPI',
    'UPi': 'UPI',
    'Upi': 'UPI',
    'Bank Transfer': 'Bank Transfer',
    'bank transfer': 'Bank Transfer',
    'BankTransfer': 'Bank Transfer',
    'Bank_Transfer': 'Bank Transfer',
    'Bank Transfr': 'Bank Transfer',
}

df['payment_mode'] = df['payment_mode'].replace(payment_mode_corrections)
df['payment_mode'] = df['payment_mode'].fillna('Unknown')

print("After cleaning payment_mode:", df['payment_mode'].unique())


After cleaning payment_mode: ['Card' 'Unknown' 'Cash' 'UPI' 'Bank Transfer']


In [53]:
print("Missing values count:")
print(df[['notes', 'location']].isna().sum())


Missing values count:
notes       2730
location    1229
dtype: int64


In [54]:
df['notes'] = df['notes'].fillna('No notes')
df['location'] = df['location'].fillna('Unknown')

print("Missing values after filling:")
print(df[['notes', 'location']].isna().sum())


Missing values after filling:
notes       0
location    0
dtype: int64


In [55]:
print("Amount column type:", df['amount'].dtype)

df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

print("Missing or invalid amounts:", df['amount'].isna().sum())

print("Zero or negative amounts:", (df['amount'] <= 0).sum())

df = df[df['amount'] > 0].reset_index(drop=True)

print("Rows after cleaning amounts:", len(df))


Amount column type: object
Missing or invalid amounts: 3351
Zero or negative amounts: 188
Rows after cleaning amounts: 11875


In [56]:
df = df[df['amount'].notna()].reset_index(drop=True)
print("Rows after dropping missing amounts:", len(df))



Rows after dropping missing amounts: 11875


In [57]:
df.to_csv('dataset_cleaned.csv', index=False)
